In [26]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv("data_for_supervised_learning.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9841 entries, 0 to 9840
Data columns (total 41 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   FLAG                                                  9841 non-null   int64  
 1   Avg-min-between-sent-tnx                              9841 non-null   float64
 2   Avg-min-between-received-tnx                          9841 non-null   float64
 3   Time-Diff-between-first-and-last (Mins)               9841 non-null   float64
 4   Sent-tnx                                              9841 non-null   int64  
 5   Received-Tnx                                          9841 non-null   int64  
 6   Number-of-Created-Contracts                           9841 non-null   int64  
 7   Unique-Received-From-Addresses                        9841 non-null   int64  
 8   Unique-Sent-To-Addresses                              9841

In [5]:
categories = df.select_dtypes(include='object').columns
categories

Index(['ERC20-most-sent-token-type', 'ERC20_most_rec_token_type'], dtype='object')

In [9]:
X = df.drop('FLAG', axis=1)
y = df['FLAG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)

In [10]:
encoder = TargetEncoder(cols=categories)
X_train_encoded = encoder.fit_transform(X_train, y_train)
X_test_encoded = encoder.transform(X_test)

In [20]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7872 entries, 4886 to 3582
Data columns (total 40 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Avg-min-between-sent-tnx                              7872 non-null   float64
 1   Avg-min-between-received-tnx                          7872 non-null   float64
 2   Time-Diff-between-first-and-last (Mins)               7872 non-null   float64
 3   Sent-tnx                                              7872 non-null   int64  
 4   Received-Tnx                                          7872 non-null   int64  
 5   Number-of-Created-Contracts                           7872 non-null   int64  
 6   Unique-Received-From-Addresses                        7872 non-null   int64  
 7   Unique-Sent-To-Addresses                              7872 non-null   int64  
 8   min-value-received                                    7

In [13]:
train_tf = tf.data.Dataset.from_tensor_slices((X_train_encoded.values, y_train.values))

In [ ]:
# Extra step to see if we can fix the loss issue
scaler = StandardScaler()
train = scaler.fit_transform(X_train_encoded)
test = scaler.transform(X_test_encoded)

In [31]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_encoded.shape[1],)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(32, activation="relu"))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))


model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

model.fit(X_train_encoded, y_train, epochs=5)

Epoch 1/5
246/246 [==============================] - 0s 762us/step - loss: nan - accuracy: 0.7755
Epoch 2/5
246/246 [==============================] - 0s 755us/step - loss: nan - accuracy: 0.7768
Epoch 3/5
246/246 [==============================] - 0s 740us/step - loss: nan - accuracy: 0.7768
Epoch 4/5
246/246 [==============================] - 0s 732us/step - loss: nan - accuracy: 0.7768
Epoch 5/5
246/246 [==============================] - 0s 740us/step - loss: nan - accuracy: 0.7768
